The `pyrit.memory` module provides functionality to keep track of the conversation history. In a nutshell, this can be used as follows

In [1]:

from uuid import uuid4
from pyrit.memory import DuckDBMemory
from pyrit.models import ChatMessage

conversation_id = str(uuid4())

memory = DuckDBMemory()
message_list = [
    ChatMessage(role="user", content="Hi, chat bot! This is my initial prompt."),
    ChatMessage(role="assistant", content="Nice to meet you! This is my response."),
]
next_message = ChatMessage(role="user", content="Wonderful! This is my second prompt to the chat bot.")
message_list.append(next_message)
memory.add_chat_messages_to_memory(conversations=message_list, conversation_id=conversation_id)


# To retrieve the items from memory

memory.get_chat_messages_with_conversation_id(conversation_id=conversation_id)

[ChatMessage(role='user', content='Hi, chat bot! This is my initial prompt.', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='Nice to meet you! This is my response.', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='Wonderful! This is my second prompt to the chat bot.', name=None, tool_calls=None, tool_call_id=None)]

In [2]:
conversation_id = str(uuid4())

memory = DuckDBMemory()
message_list = [
    ChatMessage(role="user", content="Hi, chat bot! This is my initial prompt."),
    ChatMessage(role="assistant", content="Nice to meet you! This is my response."),
]
next_message = ChatMessage(role="user", content="Wonderful! This is my second prompt to the chat bot.")
message_list.append(next_message)
memory.add_chat_messages_to_memory(conversations=message_list, conversation_id=conversation_id)


In [3]:
# To retrieve the items from memory
memory.get_chat_messages_with_conversation_id(conversation_id="11111")

[ChatMessage(role='user', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updtaed field', name=None, tool_calls=None, tool_call_id=None)]

In [4]:
# update based on conversation_id
update_fileds = {"content": "this is updated field"}
memory.update_entries_by_conversation_id(conversation_id="11111", update_fields=update_fileds)


True

In [5]:
# To retrieve the items from memory
memory.get_chat_messages_with_conversation_id(conversation_id="11111")

[ChatMessage(role='user', content='this is updated field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='this is updated field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updated field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updated field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='this is updated field', name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content='this is updated field', name=None, tool_calls=None, tool_call_id=None)]

In [6]:
# Cleanup memory resources
memory.dispose_engine()